# Acrobot LQR Balancing

This notebook demonstrates stabilization of the Acrobot model with LQR. 

*Note:* This will send draw commands to the Drake Visualizer, but you'll need to open it yourself from a separate terminal with `drake-visualizer`. 

In [1]:
# Add the current directory to the package search path, so we can "using Acrobots" later
push!(LOAD_PATH, ".");

In [2]:
using Acrobots

In [3]:
# Construct the robot model and visualizer
robot = Acrobots.acrobot()
vis = Acrobots.Visualizer(robot);

In [4]:
# Choose the initial state and desired (vertical) state
x0 = Acrobots.AcrobotState{Float64}([[pi; 0] + 0.1*(rand(2)-0.5); zeros(2)]) 
xdes = Acrobots.AcrobotState{Float64}(pi, 0, 0, 0)

Acrobots.AcrobotState{Float64}(3.141592653589793,0.0,0.0,0.0)

In [5]:
using Interact
@manipulate for x1 = linspace(-2pi, 2pi, 101), x2 = linspace(-2pi, 2pi, 101)
    Acrobots.draw(vis, Acrobots.AcrobotPosition(x1, x2))
end

Interact.Slider{Float64}(Signal{Float64}(0.0, nactions=0),"x1",0.0,linspace(-6.283185307179586,6.283185307179586,101),true)

Interact.Slider{Float64}(Signal{Float64}(0.0, nactions=0),"x2",0.0,linspace(-6.283185307179586,6.283185307179586,101),true)

nothing

In [6]:
# Linearize the system around the desired pose and 
# construct the LQR controller for that linear system
input = zero(Acrobots.AcrobotInput{Float64})
linear_sys = Acrobots.linearize(robot, 0, xdes, input)
Qf = diagm([1.,1,1,1])
Rf = eye(1)
controller = Acrobots.lqr(linear_sys, Qf, Rf, xdes)
controller_state = zero(Acrobots.LQRState{Float64})
@assert length(controller_state) == 0 # LQR controller has no state

In [7]:
# Simulate the system and controller for 5 seconds. The simulation will 
# run as fast as possible, but it's slowed down substantially by the call
# to Acrobots.draw() at every tick. Remove that line to see how fast it 
# can go. 
dt = 0.001
ts = 0:dt:5

state = x0
for (i, t) in enumerate(ts)
    output = Acrobots.output(robot, t, state, input)
    input = Acrobots.output(controller, t, controller_state, output)
    statedot = Acrobots.dynamics(robot, t, state, input)
    state += statedot * dt
    
    Acrobots.draw(vis, state)
end